In [1]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
#from time import time
#from torch import optim

In [2]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w2:',true_w2)
    X2_before_ =torch.from_numpy(np.random.normal(0, 1, (num_example, 1)))
    y_ = torch.exp(true_w2[0]*X2_before_)*torch.sin(true_w2[1]*X2_before_)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    conditionnumber=np.linalg.cond(X2_.numpy())
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.exp(true_w2[0]*X2_before_)*torch.sin(true_w2[1]*X2_before_))/torch.norm(y_))
    return y_,X2_,true_w2,P,error_reg,conditionnumber

In [3]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if showinitialw:
        print('initial w2:',w2)
    return w2

In [4]:
def sinkhorn_stabilized(a, b, M, reg, numItermax=1000, tau=1e3, stopThr=1e-9,
                        warmstart=None, verbose=False, print_period=20,
                        log=False, **kwargs):

#     a = np.asarray(a, dtype=np.float64)
#     b = np.asarray(b, dtype=np.float64)
#     M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M

#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # test if multiple target
#     if len(b.shape) > 1:
#         n_hists = b.shape[1]
#         a = a[:, np.newaxis]
#     else:
#         n_hists = 0
    n_hists = 0
    # init data
    dim_a = len(a)
    dim_b = len(b)

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    if n_hists:
        u = torch.ones((dim_a, n_hists)) / dim_a
        v = torch.ones((dim_b, n_hists)) / dim_b
    else:
        u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))- beta.reshape((1, dim_b))) / reg)

    def get_Gamma(alpha, beta, u, v):
        """log space gamma computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1)) - beta.reshape((1, dim_b)))
                      / reg + torch.log(u.reshape((dim_a, 1))) + torch.log(v.reshape((1, dim_b))))

    # print(np.min(K))

    K = get_K(alpha, beta)
    transp = K
    loop = 1
    cpt = 0
    err = 1
    while loop:

        uprev = u
        vprev = v
        # sinkhornrn update
        v = b / (torch.mm(K.transpose(1,0), u) + 1e-16)
        u = a / (torch.mm(K, v) + 1e-16)
        # remove numerical problems and store them in K
        if torch.abs(u).max() > tau or torch.abs(v).max() > tau:
            if n_hists:
                alpha, beta = alpha + reg * \
                    torch.max(torch.log(u), 1), beta + reg * torch.max(np.log(v))
            else:
                alpha, beta = alpha + reg * torch.log(u), beta + reg * torch.log(v)
                if n_hists:
                    u, v = torch.ones((dim_a, n_hists)) / dim_a, torch.ones((dim_b, n_hists)) / dim_b
                else:
                    u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b
            K = get_K(alpha, beta)
            

        if cpt % print_period == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                err_u = abs(u - uprev).max()
                err_u /= max(abs(u).max(), abs(uprev).max(), 1.)
                err_v = abs(v - vprev).max()
                err_v /= max(abs(v).max(), abs(vprev).max(), 1.)
                err = 0.5 * (err_u + err_v)
            else:
                transp = get_Gamma(alpha, beta, u, v)
                err = torch.norm((torch.sum(transp, axis=0) - b))
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 20) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr:
            loop = False

        if cpt >= numItermax:
            loop = False

        if np.any(np.isnan(u.detach().numpy())) or np.any(np.isnan(v.detach().numpy())):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break

        cpt = cpt + 1
    #print(cpt)
    if log:
        if n_hists:
            alpha = alpha[:, None]
            beta = beta[:, None]
        logu = alpha / reg + torch.log(u)
        logv = beta / reg + torch.log(v)
        log['logu'] = logu
        log['logv'] = logv
        log['alpha'] = alpha + reg * torch.log(u)
        log['beta'] = beta + reg * torch.log(v)
        log['warmstart'] = (log['alpha'], log['beta'])
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res, log

        else:
            return get_Gamma(alpha, beta, u, v), log
    else:
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res
        else:
            return get_Gamma(alpha, beta, u, v)

In [5]:
def sinkhorn_epsilon_scaling(a, b, M, reg, numItermax=100, epsilon0=1e4,
                             numInnerItermax=100, tau=1e3, stopThr=1e-9,
                             warmstart=None, verbose=False, print_period=10,
                             log=False, **kwargs):
    #a = np.asarray(a, dtype=np.float64)
    #b = np.asarray(b, dtype=np.float64)
    #M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M
#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    #dim_a=num_example
    dim_b = len(b)
    #dim_b=num_example
    # nrelative umerical precision with 64 bits
    numItermin = 35
    numItermax = max(numItermin, numItermax)  # ensure that last velue is exact

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))
                        - beta.reshape((1, dim_b))) / reg)

    # print(np.min(K))
    def get_reg(n):  # exponential decreasing
        return (epsilon0 - reg) * np.exp(-n) + reg

    loop = 1
    cpt = 0
    err = 1
    while loop:

        regi = get_reg(cpt)

        G, logi = sinkhorn_stabilized(a, b, M, regi,
                                      numItermax=numInnerItermax, stopThr=1e-9,
                                      warmstart=(alpha, beta), verbose=False,
                                      print_period=20, tau=tau, log=True)

        alpha = logi['alpha']
        beta = logi['beta']

        if cpt >= numItermax:
            loop = False

        if cpt % (print_period) == 0:  # spsion nearly converged
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            transp = G
            err = torch.norm(
                (torch.sum(transp, axis=0) - b))**2 + torch.norm((torch.sum(transp, axis=1) - a))**2
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 10) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr and cpt > numItermin:
            loop = False

        cpt = cpt + 1
    # print('err=',err,' cpt=',cpt)
    if log:
        log['alpha'] = alpha
        log['beta'] = beta
        log['warmstart'] = (log['alpha'], log['beta'])
        return G, log
    else:
        return G
    

In [7]:
gama_=1
eta=0
starts=1
for i_____ in range(5):
    for num_example in range(50,51,100): 
        for num_X2feature in [2]:
            for i____ in range(10):
                (y_,X2_,true_w2,true_P,error_reg1,conditionnumber)=generatedata(noise=0.1)
                y=y_
                X2=X2_
                results_Loss = []
                results_w2=[]
                results_error=[]
                for i__ in range(starts):
    #                     P_array=np.random.permutation(num_example)
    #                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
    #                     for i in range(num_example):
    #                         P[i][P_array[i]]=1
    #                     X_=torch.cat([X1,X2],1)
    #                     X=torch.mm(P,X_)
    #                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
    #                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    #                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
    #                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                    w2=generateinitialw(method='zeros')
                    #w2=true_w2
                    w2.requires_grad_(requires_grad=True)
    #                 results_Loss = []
                    lr=0.001
                    results_S=[]
                    t=0
                    before1=0
                    while True:                     
                        Y1=y
                        Y2=torch.exp(w2[0]*X2)*torch.sin(w2[1]*X2)
                        C=torch.zeros(num_example,num_example,dtype=torch.float64)
                        for i in range(num_example):
                            for j in range(num_example):
                                C[i][j]=(Y1[i]-Y2[j])**2            

                        #S=SinkhornIPOT(C)
                        a=torch.ones(num_example,1,dtype=torch.float64)
                        b=torch.ones(num_example,1,dtype=torch.float64)
                        S=sinkhorn_epsilon_scaling(a, b, C, 0.00000001)
                        #print(S.transpose(1,0).half())
                        #results_S.append(S)
                        #if t>0:
                            #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                        #Loss=torch.sum(S*C)
                        Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                        if Loss<1e-2:
                            break
                        Loss.backward()
    #                         results_Loss.append(Loss)
    #                         for i_ in range(num_X1features):
    #                             results_w1[t][i_]=(w1[i_].data)
    #                         for i_ in range(num_X2features):
    #                             results_w2[t][i_]=(w2[i_].data)
                        w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                        #print(w2.grad)
    #                     if t==num_epochs-1:
    #                         print('最终w1梯度：',w1.grad)
    #                         print('最终w2梯度：',w2.grad)
                        w2.grad.data.zero_() 
                        print('Loss',t,'={:.4f}'.format(Loss.data))
                        #print('Loss',t,'=',Loss)
    #                     if t%6==0:
    #                         if torch.norm(Loss-before1)<1e-4:
    #                             break
    #                         before1=Loss
                        if torch.norm(Loss-before1)/before1<3e-5:
                            break
                        before1=Loss
                        if t>=200:
                            print('超过迭代上限')
                            break
                        if math.isnan(Loss):
                            break
                        t+=1
                        print('#',end='')



                    print(' ',end='')
                    error_each=(torch.norm(w2-true_w2))
                    #results_error.append(error_each)
                    error_each2=(torch.norm(w2-true_w2))/torch.norm(true_w2)
                    
                    #results_Loss.append(Loss)
                    #results_w1.append(w1.data)
                    #results_w2.append(w2.data)


                #w1=results_w1[results_Loss.index(min(results_Loss))]
                #w2=results_w2[results_Loss.index(min(results_Loss))]

    #                     for i_ in range(starts):
    #                         results_w1[i_]=(w1[i_].data)
    #                     for i_ in range(starts):
    #                         results_w2[i_]=(w2[i_].data)


                #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
                #print(num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')
                print(num_X2feature,num_example,'绝对误差',error_each,end='   ')
                print('相对误差',error_each2,end='   ')
                #print('真实置换矩阵为：',true_P)    
                
                error_reg2=(torch.norm(y_-torch.exp(w2[0]*torch.mm(S,X2_))*torch.sin(w2[1]*torch.mm(S,X2_)))/torch.norm(y_))
                print('真实回归误差',error_reg1,end='   ')
                print('实验回归误差',error_reg2,end='   ')
                error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
                print('置换矩阵误差',error_P,end='   ')
                matrix_count=0
                for i_2 in range(num_example):
                    for j_2 in range(num_example):
                        if (abs(S[i_2][j_2]-1)<0.02):
                            matrix_count+=1
                print('离置换矩阵距离',matrix_count,end='   ')
                print('条件数',conditionnumber)
                #print('双随机矩阵S为：',S.transpose(1,0).half())
                #print(results)
    #                 plt.figure(figsize=(6,6))
    #                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
    #                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
    #                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
    #                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
    #                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
    #                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
    #                 plt.legend()
    #                 plt.show()

Loss 0 =26.2376
#Loss 1 =26.1318
#Loss 2 =20.0777
#Loss 3 =15.7599
#Loss 4 =12.6989
#Loss 5 =10.4896
#Loss 6 =8.8455
#Loss 7 =7.5795
#Loss 8 =6.5729
#Loss 9 =5.7509
#Loss 10 =5.0657
#Loss 11 =4.4862
#Loss 12 =3.9910
#Loss 13 =3.5651
#Loss 14 =3.1973
#Loss 15 =2.8762
#Loss 16 =2.5938
#Loss 17 =2.3412
#Loss 18 =2.1218
#Loss 19 =1.9308
#Loss 20 =1.7649
#Loss 21 =1.6207
#Loss 22 =1.4950
#Loss 23 =1.3813
#Loss 24 =1.2798
#Loss 25 =1.1892
#Loss 26 =1.1094
#Loss 27 =1.0391
#Loss 28 =0.9764
#Loss 29 =0.9214
#Loss 30 =0.8737
#Loss 31 =0.8319
#Loss 32 =0.7947
#Loss 33 =0.7616
#Loss 34 =0.7298
#Loss 35 =0.7019
#Loss 36 =0.6774
#Loss 37 =0.6557
#Loss 38 =0.6364
#Loss 39 =0.6193
#Loss 40 =0.6039
#Loss 41 =0.5891
#Loss 42 =0.5737
#Loss 43 =0.5582
#Loss 44 =0.5443
#Loss 45 =0.5317
#Loss 46 =0.5202
#Loss 47 =0.5095
#Loss 48 =0.4994
#Loss 49 =0.4901
#Loss 50 =0.4817
#Loss 51 =0.4732
#Loss 52 =0.4643
#Loss 53 =0.4560
#Loss 54 =0.4484
#Loss 55 =0.4413
#Loss 56 =0.4348
#Loss 57 =0.4287
#Loss 58 =0.4231
#L

#Loss 27 =80.9440
#Loss 28 =77.7702
#Loss 29 =74.3230
#Loss 30 =70.9956
#Loss 31 =68.2452
#Loss 32 =66.3955
#Loss 33 =65.3900
#Loss 34 =64.8923
#Loss 35 =64.6009
#Loss 36 =64.3917
#Loss 37 =64.2277
#Loss 38 =64.0957
#Loss 39 =63.9879
#Loss 40 =63.8991
#Loss 41 =63.8252
#Loss 42 =63.7632
#Loss 43 =63.7108
#Loss 44 =63.6663
#Loss 45 =63.6282
#Loss 46 =63.5954
#Loss 47 =63.5670
#Loss 48 =63.5424
#Loss 49 =63.5209
#Loss 50 =63.5022
#Loss 51 =63.4857
#Loss 52 =63.4711
#Loss 53 =63.4583
#Loss 54 =63.4469
#Loss 55 =63.4368
#Loss 56 =63.4277
#Loss 57 =63.4197
#Loss 58 =63.4125
#Loss 59 =63.4060
#Loss 60 =63.4002
#Loss 61 =63.3950
#Loss 62 =63.3903
#Loss 63 =63.3861
#Loss 64 =63.3822
#Loss 65 =63.3788
#Loss 66 =63.3756
#Loss 67 =63.3728
#Loss 68 =63.3702
#Loss 69 =63.3678
#Loss 70 =63.3657
#Loss 71 =63.3637
#Loss 72 =63.3620
 2 50 绝对误差 tensor(2.2307, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(1.1132, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0537, dtype=tor

#Loss 182 =0.6502
#Loss 183 =0.6493
#Loss 184 =0.6484
#Loss 185 =0.6476
#Loss 186 =0.6468
#Loss 187 =0.6460
#Loss 188 =0.6453
#Loss 189 =0.6446
#Loss 190 =0.6439
#Loss 191 =0.6432
#Loss 192 =0.6426
#Loss 193 =0.6420
#Loss 194 =0.6414
#Loss 195 =0.6408
#Loss 196 =0.6403
#Loss 197 =0.6398
#Loss 198 =0.6393
#Loss 199 =0.6388
#Loss 200 =0.6383
超过迭代上限
 2 50 绝对误差 tensor(0.8083, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(0.4362, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1546, dtype=torch.float64)   实验回归误差 tensor(0.1720, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 50   条件数 1.0
Loss 0 =768.6010
#Loss 1 =765.8251
#Loss 2 =712.8282
#Loss 3 =662.7824
#Loss 4 =592.4804
#Loss 5 =460.1954
#Loss 6 =148.3280
#Loss 7 =1094.3681
#Loss 8 =700.6016
#Loss 9 =632.6594
#Loss 10 =680.5202
#Loss 11 =660.8483
#Loss 12 =681.7491
#Loss 13 =705.3146
#Loss 14 =667.9935
#Loss 15 =671.3782
#Loss 16 =62

#Loss 186 =7.8830
#Loss 187 =3.7606
#Loss 188 =7.8830
#Loss 189 =3.7606
#Loss 190 =7.8830
#Loss 191 =3.7606
#Loss 192 =7.8830
#Loss 193 =3.7606
#Loss 194 =7.8830
#Loss 195 =3.7606
#Loss 196 =7.8830
#Loss 197 =3.7606
#Loss 198 =7.8830
#Loss 199 =3.7606
#Loss 200 =7.8830
超过迭代上限
 2 50 绝对误差 tensor(0.0719, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(0.0825, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0598, dtype=torch.float64)   实验回归误差 tensor(0.1605, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.1832, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 50   条件数 1.0
Loss 0 =16.6970
#Loss 1 =16.6292
#Loss 2 =13.8026
#Loss 3 =11.5375
#Loss 4 =9.7189
#Loss 5 =8.2480
#Loss 6 =7.0451
#Loss 7 =6.0490
#Loss 8 =5.2139
#Loss 9 =4.5061
#Loss 10 =3.9007
#Loss 11 =3.3797
#Loss 12 =2.9297
#Loss 13 =2.5405
#Loss 14 =2.2043
#Loss 15 =1.9148
#Loss 16 =1.6667
#Loss 17 =1.4553
#Loss 18 =1.2765
#Loss 19 =1.1264
#Loss 20 =1.0014
#Loss 21 =0.8981
#Loss 22 =0

#Loss 59 =6.0186
#Loss 60 =5.9868
#Loss 61 =5.9559
#Loss 62 =5.9259
#Loss 63 =5.8966
#Loss 64 =5.8680
#Loss 65 =5.8402
#Loss 66 =5.8132
#Loss 67 =5.7868
#Loss 68 =5.7610
#Loss 69 =5.7359
#Loss 70 =5.7114
#Loss 71 =5.6875
#Loss 72 =5.6641
#Loss 73 =5.6413
#Loss 74 =5.6190
#Loss 75 =5.5971
#Loss 76 =5.5758
#Loss 77 =5.5550
#Loss 78 =5.5346
#Loss 79 =5.5146
#Loss 80 =5.4951
#Loss 81 =5.4760
#Loss 82 =5.4573
#Loss 83 =5.4390
#Loss 84 =5.4211
#Loss 85 =5.4035
#Loss 86 =5.3863
#Loss 87 =5.3694
#Loss 88 =5.3528
#Loss 89 =5.3366
#Loss 90 =5.3206
#Loss 91 =5.3050
#Loss 92 =5.2897
#Loss 93 =5.2747
#Loss 94 =5.2599
#Loss 95 =5.2454
#Loss 96 =5.2312
#Loss 97 =5.2173
#Loss 98 =5.2036
#Loss 99 =5.1901
#Loss 100 =5.1769
#Loss 101 =5.1639
#Loss 102 =5.1512
#Loss 103 =5.1386
#Loss 104 =5.1263
#Loss 105 =5.1142
#Loss 106 =5.1023
#Loss 107 =5.0906
#Loss 108 =5.0791
#Loss 109 =5.0677
#Loss 110 =5.0566
#Loss 111 =5.0456
#Loss 112 =5.0348
#Loss 113 =5.0242
#Loss 114 =5.0138
#Loss 115 =5.0035
#Loss 116 =4.99

...

#Loss 65 =1.2150
#Loss 66 =1.2113
#Loss 67 =1.2079
#Loss 68 =1.2043
#Loss 69 =1.2011
#Loss 70 =1.1979
#Loss 71 =1.1941
#Loss 72 =1.1905
#Loss 73 =1.1872
#Loss 74 =1.1842
#Loss 75 =1.1813
#Loss 76 =1.1781
#Loss 77 =1.1752
#Loss 78 =1.1726
#Loss 79 =1.1701
#Loss 80 =1.1679
#Loss 81 =1.1659
#Loss 82 =1.1640
#Loss 83 =1.1623
#Loss 84 =1.1607
#Loss 85 =1.1592
#Loss 86 =1.1576
#Loss 87 =1.1556
#Loss 88 =1.1536
#Loss 89 =1.1517
#Loss 90 =1.1499
#Loss 91 =1.1480
#Loss 92 =1.1463
#Loss 93 =1.1447
#Loss 94 =1.1432
#Loss 95 =1.1418
#Loss 96 =1.1406
#Loss 97 =1.1394
#Loss 98 =1.1384
#Loss 99 =1.1374
#Loss 100 =1.1362
#Loss 101 =1.1352
#Loss 102 =1.1343
#Loss 103 =1.1334
#Loss 104 =1.1327
#Loss 105 =1.1319
#Loss 106 =1.1313
#Loss 107 =1.1307
#Loss 108 =1.1301
#Loss 109 =1.1296
#Loss 110 =1.1291
#Loss 111 =1.1285
#Loss 112 =1.1280
#Loss 113 =1.1275
#Loss 114 =1.1270
#Loss 115 =1.1266
#Loss 116 =1.1262
#Loss 117 =1.1259
#Loss 118 =1.1256
#Loss 119 =1.1253
#Loss 120 =1.1250
#Loss 121 =1.1248
#Loss 122

#Loss 23 =0.8027
#Loss 24 =0.7476
#Loss 25 =0.6988
#Loss 26 =0.6555
#Loss 27 =0.6167
#Loss 28 =0.5820
#Loss 29 =0.5506
#Loss 30 =0.5222
#Loss 31 =0.4964
#Loss 32 =0.4729
#Loss 33 =0.4513
#Loss 34 =0.4314
#Loss 35 =0.4131
#Loss 36 =0.3962
#Loss 37 =0.3806
#Loss 38 =0.3661
#Loss 39 =0.3525
#Loss 40 =0.3399
#Loss 41 =0.3282
#Loss 42 =0.3172
#Loss 43 =0.3069
#Loss 44 =0.2973
#Loss 45 =0.2882
#Loss 46 =0.2798
#Loss 47 =0.2718
#Loss 48 =0.2643
#Loss 49 =0.2573
#Loss 50 =0.2507
#Loss 51 =0.2444
#Loss 52 =0.2386
#Loss 53 =0.2330
#Loss 54 =0.2278
#Loss 55 =0.2229
#Loss 56 =0.2183
#Loss 57 =0.2140
#Loss 58 =0.2098
#Loss 59 =0.2060
#Loss 60 =0.2023
#Loss 61 =0.1989
#Loss 62 =0.1956
#Loss 63 =0.1925
#Loss 64 =0.1896
#Loss 65 =0.1869
#Loss 66 =0.1843
#Loss 67 =0.1818
#Loss 68 =0.1795
#Loss 69 =0.1773
#Loss 70 =0.1753
#Loss 71 =0.1733
#Loss 72 =0.1715
#Loss 73 =0.1697
#Loss 74 =0.1681
#Loss 75 =0.1665
#Loss 76 =0.1651
#Loss 77 =0.1637
#Loss 78 =0.1623
#Loss 79 =0.1611
#Loss 80 =0.1599
#Loss 81 =0.15

#Loss 65 =2.1307
#Loss 66 =2.1284
#Loss 67 =2.1262
#Loss 68 =2.1241
#Loss 69 =2.1220
#Loss 70 =2.1201
#Loss 71 =2.1182
#Loss 72 =2.1164
#Loss 73 =2.1146
#Loss 74 =2.1129
#Loss 75 =2.1113
#Loss 76 =2.1097
#Loss 77 =2.1082
#Loss 78 =2.1068
#Loss 79 =2.1054
#Loss 80 =2.1040
#Loss 81 =2.1027
#Loss 82 =2.1015
#Loss 83 =2.1002
#Loss 84 =2.0991
#Loss 85 =2.0980
#Loss 86 =2.0969
#Loss 87 =2.0958
#Loss 88 =2.0948
#Loss 89 =2.0938
#Loss 90 =2.0929
#Loss 91 =2.0920
#Loss 92 =2.0911
#Loss 93 =2.0903
#Loss 94 =2.0895
#Loss 95 =2.0887
#Loss 96 =2.0879
#Loss 97 =2.0872
#Loss 98 =2.0865
#Loss 99 =2.0858
#Loss 100 =2.0851
#Loss 101 =2.0845
#Loss 102 =2.0839
#Loss 103 =2.0833
#Loss 104 =2.0827
#Loss 105 =2.0821
#Loss 106 =2.0816
#Loss 107 =2.0811
#Loss 108 =2.0806
#Loss 109 =2.0801
#Loss 110 =2.0796
#Loss 111 =2.0792
#Loss 112 =2.0787
#Loss 113 =2.0783
#Loss 114 =2.0779
#Loss 115 =2.0775
#Loss 116 =2.0771
#Loss 117 =2.0767
#Loss 118 =2.0764
#Loss 119 =2.0760
#Loss 120 =2.0757
#Loss 121 =2.0754
#Loss 122

#Loss 92 =2.2906
#Loss 93 =2.2421
#Loss 94 =2.1969
#Loss 95 =2.1547
#Loss 96 =2.1153
#Loss 97 =2.0784
#Loss 98 =2.0439
#Loss 99 =2.0116
#Loss 100 =1.9814
#Loss 101 =1.9531
#Loss 102 =1.9265
#Loss 103 =1.9016
#Loss 104 =1.8782
#Loss 105 =1.8563
#Loss 106 =1.8357
#Loss 107 =1.8164
#Loss 108 =1.7984
#Loss 109 =1.7819
#Loss 110 =1.7677
#Loss 111 =1.7582
#Loss 112 =1.7611
#Loss 113 =1.8031
#Loss 114 =1.9676
#Loss 115 =2.5729
#Loss 116 =4.5211
#Loss 117 =11.6004
#Loss 118 =25.1691
#Loss 119 =54.0433
#Loss 120 =8.8821
#Loss 121 =5.3543
#Loss 122 =3.5405
#Loss 123 =3.2723
#Loss 124 =3.1202
#Loss 125 =3.0048
#Loss 126 =2.9028
#Loss 127 =2.8099
#Loss 128 =2.7243
#Loss 129 =2.6452
#Loss 130 =2.5719
#Loss 131 =2.5039
#Loss 132 =2.4407
#Loss 133 =2.3820
#Loss 134 =2.3273
#Loss 135 =2.2763
#Loss 136 =2.2288
#Loss 137 =2.1845
#Loss 138 =2.1431
#Loss 139 =2.1044
#Loss 140 =2.0683
#Loss 141 =2.0345
#Loss 142 =2.0029
#Loss 143 =1.9734
#Loss 144 =1.9459
#Loss 145 =1.9204
#Loss 146 =1.8972
#Loss 147 =1.87

 2 50 绝对误差 tensor(0.0211, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(0.0379, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1967, dtype=torch.float64)   实验回归误差 tensor(0.1027, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.2490, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 50   条件数 1.0
Loss 0 =5516.4861
#Loss 1 =5515.3859
#Loss 2 =5262.8973
#Loss 3 =4835.8591
#Loss 4 =110.3886
#Loss 5 =4187.1993
#Loss 6 =78503.3613
#Loss 7 =nan
 2 50 绝对误差 tensor(nan, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(nan, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0098, dtype=torch.float64)   实验回归误差 tensor(nan, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.0000, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 0   条件数 1.0
Loss 0 =317.6623
#Loss 1 =316.3779
#Loss 2 =289.5972
#Loss 3 =266.4510
#Loss 4 =239.5698
#Loss 5 =201.9294
#Loss 6 =143.3468
#Loss 7 =56.0893
#Loss 8 =35.1009
#Loss 9 =69.2222